<a href="https://colab.research.google.com/github/viett887g/2023-10-27-baitapML/blob/main/redo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [119]:
import numpy as np
import os
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd

# **I. Chọn dataset :USA Housing**

In [120]:
# Hàm đọc dữ liệu từ file
def readdata(file):
    data = pd.read_csv(file)
    del data['Address']
    data = np.array(data)
    return data

# Hàm chuẩn hóa dữ liệu và tạo training set và test set
def create_data(data):
    data = preprocessing.MinMaxScaler().fit_transform(data)
    X = data[:,:-1]
    y = data[:, -1]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test

# Hàm sigmoid
def sigmoid(X, w):
    Hw = 1 / (1 + np.exp(-np.dot(X, w)))
    return Hw

# Hàm tính giá trị los
def loss(w, X, y):
    m = X.shape[0]
    h = sigmoid(X, w)
    result = (-1 / m) * np.sum(np.dot(y.T, np.log(h)) + np.dot((1 - y).T, np.log(1 - h)))
    return result

# II. Chọn 2 phương pháp **Gradient Descent (GD)**, **Momentum Based GD** tối ưu để tối ưu hoá loss function Linear Regression cho tập dữ liệu USA Housing

##**2.1 Gradient Descent (GD)**

In [121]:
# Hàm gradient descent
def gradient_descent(X, y, w, learning_rate, num_iterations):
    m = X.shape[0]
    loss_history = []

    for i in range(num_iterations):
        h = sigmoid(X, w)
        gradient = (1 / m) * np.dot(X.T, (h - y))
        w = w - learning_rate * gradient
        loss_value = loss(w, X, y)
        loss_history.append(loss_value)

    return w, loss_history

## **2.2 Momentum Based GD**

In [122]:
def momentum_based_gradient_descent(X, y, w, learning_rate, num_iterations, momentum):
    m = X.shape[0]
    loss_history = []
    velocity = np.zeros_like(w)

    for i in range(num_iterations):
        h = sigmoid(X, w)
        gradient = (1 / m) * np.dot(X.T, (h - y))

        velocity = momentum * velocity + learning_rate * gradient
        w = w - velocity

        loss_value = loss(w, X, y)
        loss_history.append(loss_value)

    return w, loss_history

# **III. So sánh hiệu suất của các thuật toán**

In [123]:
def evaluate_performance(X_test, y_test, w):
    # Tính toán dự đoán trên tập test
    y_pred = np.dot(X_test, w)

    # Tính toán Mean Squared Error (MSE)
    mse = mean_squared_error(y_test, y_pred)

    # Tính toán Rmse
    rmse = np.sqrt(mse)


    # Trả về kết quả hiệu suất của mô hình
    return mse, rmse

In [128]:
def main():
    data = readdata("/content/drive/MyDrive/USA_Housing.csv")
    X_train, X_test, y_train, y_test = create_data(data)
    # Khởi tạo vector trọng số w
    w = np.zeros(X_train.shape[1])
    # Thiết lập tỷ lệ học tập và số lần lặp
    learning_rate = 0.01
    num_iterations = 1000
    momentum = 0.9
    # Thực hiện gradient descent để tối ưu hàm loss và trả về vector trọng số tối ưu và lịch sử giá trị loss
    w_optimal, loss_history = gradient_descent(X_train, y_train, w, learning_rate, num_iterations)

    # Tính giá trị loss tối ưu trên tập test
    optimal_loss = loss(w_optimal, X_test, y_test)
    print("Giá trị loss tối ưu của Gradient Descents :", optimal_loss)
    mse, rmse = evaluate_performance(X_test, y_test, w_optimal)

    print("Mean Squared Error của Gradient Descents (MSE):", mse)
    print('Root Mean Squared Error của Gradient Descents:',rmse)
    print('\n','\n')

    w_optimal, loss_history = momentum_based_gradient_descent(X_train, y_train, w, learning_rate, num_iterations, momentum)

    optimal_loss = loss(w_optimal, X_test, y_test)
    print("Giá trị loss tối ưu của momentum_based_gradient_descent :", optimal_loss)
    mse, rmse = evaluate_performance(X_test, y_test, w_optimal)

    print("Mean Squared Error (MSE) của momentum_based_gradient_descent:", mse)
    print('Root Mean Squared Error của momentum_based_gradient_descent:',rmse)

if __name__ == "__main__":
    main()

Giá trị loss tối ưu của Gradient Descents : 0.6926051747191877
Mean Squared Error của Gradient Descents (MSE): 0.22673429024886363
Root Mean Squared Error của Gradient Descents: 0.47616624223989634

 

Giá trị loss tối ưu của momentum_based_gradient_descent : 0.692207168506447
Mean Squared Error (MSE) của momentum_based_gradient_descent: 0.22426197010903098
Root Mean Squared Error của momentum_based_gradient_descent: 0.4735630582182599


# **IV. Nhận xét**

mô hình momentum_based_gradient_descent có giá trị loss tối ưu thấp hơn, cho thấy mô hình này có hiệu suất tốt hơn trong việc tối ưu hóa hàm mất mát.

mô hình momentum_based_gradient_descent có giá trị MSE thấp hơn, cho thấy mô hình này có khả năng dự đoán chính xác hơn và đạt được mức độ biến động giữa giá trị dự đoán và giá trị thực tế tốt hơn.

mô hình momentum_based_gradient_descent có giá trị RMSE thấp hơn, cho thấy mô hình này có khả năng dự đoán chính xác hơn và có độ lệch trung bình giữa giá trị dự đoán và giá trị thực tế thấp hơn.

# **IV. Kết luận**
Dựa trên các kết quả trên, mô hình momentum_based_gradient_descent có giá trị loss thấp hơn và các chỉ số MSE và RMSE nhỏ hơn, do đó có thể xem là mô hình này tối ưu hơn và tốt hơn so với mô hình Gradient Descents.

# **V. Ưu nhược điểm của phương pháp tối ưu.**

**Ưu điểm của phương pháp tối ưu Gradient Descent (GD):**

Đơn giản và dễ hiểu: Gradient Descent là một phương pháp tối ưu đơn giản và dễ hiểu. Nó chỉ yêu cầu tính đạo hàm đơn giản của hàm mất mát, và cập nhật các tham số theo hướng giảm độ dốc của đạo hàm.

Hiệu quả tính toán: GD thường được sử dụng trong nhiều mô hình học máy công nghệ cao, bởi vì nó có thể xử lý tập dữ liệu lớn một cách hiệu quả. Việc tính toán gradient chỉ phụ thuộc vào kích thước của batch, và không yêu cầu tính toán ma trận đầy đủ.

Hội tụ tới điểm tối ưu cục bộ: Nếu được cài đặt đúng, GD có khả năng hội tụ tới điểm tối ưu cục bộ. Nếu hàm mất mát là lồi và tốt hơn, GD có thể cung cấp giải pháp tối ưu đối tượng.

**Nhược điểm của phương pháp tối ưu Gradient Descent (GD):**

Tốc độ hội tụ chậm: GD có thể hội tụ chậm đối với một số bài toán, đặc biệt là khi hàm mất mát có đặc trưng vùng trong lỏng lẻo hoặc bề mặt lõm. Việc cập nhật các tham số trên toàn bộ dữ liệu có thể cần nhiều vòng lặp để đạt điểm tối ưu.
Momentum Based Gradient Descent (GD với đà):


**Ưu điểm Momentum Based GD** có thể giúp cải thiện tốc độ hội tụ của GD. Bằng cách tích lũy đà từ các bước trước, nó có khả năng vượt qua các mục tiêu hẹp và giảm thiểu trỗi dạng cục bộ, đồng thời giảm ảnh hưởng của các điểm nhiễu trên đường đi hội tụ.

**Nhược điểm Momentum Based GD**: Mặc dù hiệu quả hơn so với GD cơ bản, tuy nhiên không phải lúc nào Momentum Based GD cũng tốt hơn. Trong một số trường hợp, điều này có thể gây ra quá đà, kéo dài thời gian hội tụ hoặc không hội tụ tới điểm tối ưu. Bên cạnh đó, việc điều chỉnh thông số đà có thể không dễ dàng và yêu cầu thử nghiệm.